# Problem 1
A static map of cycling accidents in Helsinki

In [12]:
import pandas as pd
import geopandas as gpd
from pyproj import CRS
import matplotlib.pyplot as plt
from shapely.geometry import Point
import folium

In [2]:
# Import the data, select columns that are needed
fp ="data/tieliikenneonnettomuudet_2019_onnettomuus.csv"
data = pd.read_csv(fp, delimiter=";", usecols=["Onnett_id", "X", "Y", "Vakavuusko", "Vakavuus", "Onluokka", "Kuntasel", "Ontyyppi", "Ontyypsel", "Päivä"])





In [3]:
fp2 = "data/l_tiesto.shp"
roads = gpd.read_file(fp2)

fp3 = "data/m_meri_region.shp"
sea = gpd.read_file(fp3)

In [4]:
data.dtypes

Onnett_id       int64
Päivä          object
Vakavuusko      int64
Vakavuus       object
Ontyyppi        int64
Ontyypsel      object
Onluokka        int64
Kuntasel       object
X             float64
Y             float64
dtype: object

In [5]:
data


,Onnett_id,Päivä,Vakavuusko,Vakavuus,Ontyyppi,Ontyypsel,Onluokka,Kuntasel,X,Y
0,9332527,05.02.2019,0,Ei henkilövahinkoja,8,peräänajo liikenne-esteen takia pysäht. ajoneu...,6,Helsinki,382577.0,6675844.0
1,9343914,12.06.2019,0,Ei henkilövahinkoja,81,suistuminen vasemmalle suoralla,1,Helsinki,382543.0,6675979.0
2,9398457,13.09.2019,0,Ei henkilövahinkoja,8,peräänajo liikenne-esteen takia pysäht. ajoneu...,6,Helsinki,382542.0,6675979.0
3,9398382,22.09.2019,0,Ei henkilövahinkoja,8,peräänajo liikenne-esteen takia pysäht. ajoneu...,6,Helsinki,382540.0,6675979.0
4,9533397,06.12.2019,0,Ei henkilövahinkoja,82,suistuminen oik. oikealle kääntyvässä kaarteessa,1,Helsinki,382491.0,6675945.0
...,...,...,...,...,...,...,...,...,...,...
15292,9534641,27.12.2019,2,Loukkaantumiseen johtanut,80,suistuminen oikealle suoralla,1,Muonio,361920.0,7539419.0
15293,9534611,28.12.2019,2,Loukkaantumiseen johtanut,83,suistuminen vas. oikealle kääntyvässä kaarteessa,1,Sastamala,293592.0,6811639.0
15294,9533505,29.12.2019,2,Loukkaantumiseen johtanut,96,peruutusonnettomuus,13,Vantaa,394062.0,6692018.0
15295,9533827,30.12.2019,0,Ei henkilövahinkoja,96,peruutusonnettomuus,1,Kuopio,514489.0,6973933.0


In [27]:
# Select only bicycle accidents (Onluokka=8) that have happenede in Helsinki, Espoo, Vantaa or Kauniainen

# Create a list for the municipalities on the map
municipalities = ["Helsinki", "Espoo", "Vantaa", "Kauniainen"]

# Filter the data by acciden type and municipality
data = data.loc[data["Kuntasel"].isin(municipalities)]

cycling = data.loc[data["Onluokka"]==8]
walking = data.loc[data["Onluokka"]==9]


In [28]:
walking

,Onnett_id,Päivä,Vakavuusko,Vakavuus,Ontyyppi,Ontyypsel,Onluokka,Kuntasel,X,Y
3347,9335186,03.04.2019,0,Ei henkilövahinkoja,79,muu jalankulkuonnettomuus suojatien ulkopuolella,9,Helsinki,384155.0,6671549.0
4197,9332171,27.02.2019,0,Ei henkilövahinkoja,73,jalankulkija kulki liikenteen suuntaan,9,Espoo,365328.0,6681088.0
4249,9533684,11.12.2019,2,Loukkaantumiseen johtanut,69,muu jalankulkuonnettomuus suojatiellä,9,Helsinki,380734.0,6680514.0
4352,9534506,03.12.2019,2,Loukkaantumiseen johtanut,69,muu jalankulkuonnettomuus suojatiellä,9,Vantaa,396620.0,6691336.0
5769,9387802,28.08.2019,2,Loukkaantumiseen johtanut,69,muu jalankulkuonnettomuus suojatiellä,9,Espoo,366074.0,6674746.0
...,...,...,...,...,...,...,...,...,...,...
14923,9385841,08.07.2019,2,Loukkaantumiseen johtanut,75,jk jalkakäytävällä tai liikennekorokkeella,9,Helsinki,384574.0,6674666.0
14929,9385998,10.07.2019,2,Loukkaantumiseen johtanut,79,muu jalankulkuonnettomuus suojatien ulkopuolella,9,Vantaa,392431.0,6688742.0
15119,9398776,27.09.2019,0,Ei henkilövahinkoja,96,peruutusonnettomuus,9,Helsinki,384035.0,6678294.0
15171,9477852,26.10.2019,2,Loukkaantumiseen johtanut,96,peruutusonnettomuus,9,Helsinki,393908.0,6682579.0


Turn the data into a geodataframe

In [29]:
# Use function from geopandas for the task
cycling_geo = gpd.GeoDataFrame(
    cycling, geometry=gpd.points_from_xy(cycling.X, cycling.Y), crs="EPSG:3067")

walking_geo = gpd.GeoDataFrame(
    walking, geometry=gpd.points_from_xy(walking.X, walking.Y), crs="EPSG:3067")

In [35]:
# Convert dataframes to GeoJSON
cycling_gjson = folium.features.GeoJson(cycling_geo, name="Cycling_accidents")
walking_gjson = folium.features.GeoJson(walking_geo, name="Pedestrian_accidents")

In [ ]:
# Print crs's

print(geo.crs)
print("---")
print(roads.crs)
print("---")
print(sea.crs)

In [ ]:
# Reproject files to sea.crs



In [36]:
# Create a Map instance
m = folium.Map(location=[60.2, 24.9], zoom_start=11, control_scale=True, tiles="CartoDB dark_matter")
cycling_gjson.add_to(m)
walking_gjson.add_to(m)

m